# Vehicle Routing Problem
[Murwan Siddig](mailto:msiddig@clemson.edu)

------------------------------------------------------------------------
## Pseudo-polynomial Formulation

In [1]:
include("data_VRP.jl");

In [2]:
#technical lines
using Random;
using Distributions;
using LinearAlgebra
using JuMP;
using Gurobi;
const GRB_ENV = Gurobi.Env();


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [3]:
#Function to extract the tour for each vehicle in operation 
function extractTour(n, sol)
    #Initilaize the an empty list for the lists of tours
    tours = [];
    #The number of vehicles in operation is equal to the number of arcs leaving the depot node 
    used_vehilces = sum(sol[1,j] for j=1:n);
    count = used_vehilces
    #Keep doing this loop until the tour for each vehicle in use is obtained 
    while count !=0
        count -=1
        tour = [1]  # Start at city 1 always
        cur_city = 1
        while true
            # Look for first arc out of current city
            for j = 1:n
                if sol[cur_city,j] >= 1-1e-6
                    # Found next city
                    push!(tour, j)
                    # Don't ever use this arc again
                    sol[cur_city, j] = 0.0
                    sol[j, cur_city] = 0.0
                    # Move to next city
                    cur_city = j
                    break
                end
            end
            # If we have come back to 1, stop
            if cur_city == 1
                break
            end
        end  # end while
        push!(tours, tour)
    end
    
    return tours
end

extractTour (generic function with 1 method)

In [4]:
#Pseudo-polynomial formulation 

VRP3 = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));

@variable(VRP3, x[1:nn,1:nn,0:Q], Bin);
@objective(VRP3, Min, sum(sum(c[i,j]*x[i,j,k] for k=0:Q) for i=1:nn for j=1:nn));
                
for i = 1:nn
    @constraint(VRP3, sum(x[i,i,k] for k=0:Q) == 0)
end  
                
for i=2:nn
    ind = [];
    for k=1:nn
        if k !=i
            push!(ind,k)
        end
    end
    @constraint(VRP3, sum(sum(x[i,j,k] for k=0:Q) for j in ind)== 1);
    l = q[i];
    for k=l:Q
        @constraint(VRP3, sum(x[j,i,k] for j in ind)-sum(x[i,j,(k-l)] for j in ind)== 0);
    end
end
@constraint(VRP3, sum(sum(x[1,j,k] for k=0:Q) for j=2:nn)==sum(sum(x[j,1,k] for k=0:Q) for j=2:nn));
@constraint(VRP3, sum(sum(x[j,1,k] for k=0:Q) for j=2:nn) <=nv);
@constraint(VRP3, sum(sum(x[1,j,k] for k=0:Q) for j=2:nn) <=nv);
   
for i=1:nn
    for k=1:Q
        @constraint(VRP3, x[i,1,k] ==0);
    end
    
    for j=2:nn
        if j==i
            continue
        else
            t = q[j]-1
            for k=0:t
                @constraint(VRP3, x[i,j,k] ==0);
            end
        end
    end
end

In [15]:
@time optimize!(VRP3)
println("***********************************************")
println("***********************************************")
println(" solved! ")
println("===============================================")
#tours = extractTour(nn, value.(x)[])
#println("Tours = ",tours)
println("This solution is Feasible and Optimal")
println("Optimal objective value = ", objective_value(VRP3));

***********************************************
***********************************************
 solved! 
This solution is Feasible and Optimal
Optimal objective value = 784.0
